In [44]:
from pyhive import trino
import pandas as pd
import re

# 创建连接
conn = trino.connect(
    host='ec2-18-116-88-221.us-east-2.compute.amazonaws.com',
    port=8080,
    catalog='pixels',
    schema='clickbench',
)

# 创建一个cursor对象
cursor = conn.cursor()

def calculate_cost(query, cursor):
    # 定义你的EXPLAIN查询
    query = 'EXPLAIN ' + query

    # 执行查询
    cursor.execute(query)

    # 获取结果
    rows = cursor.fetchall()

    # 将结果转为DataFrame
    df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])

    # 提取所有Scan操作的存储大小，并分解为数量和单位
    scan_costs = df[df.iloc[:, 0].str.contains('Scan')].iloc[:, 0].str.extractall(r'Estimates: \{rows: [0-9]* \((?P<rows_size>[0-9\.]*)(?P<rows_unit>[A-Za-z]*)\)')

    # 转化数量到对应的数字
    scan_costs['rows_size'] = scan_costs['rows_size'].astype(float)

    # 创建一个字典用于存储每种单位的转换系数
    unit_conversion = {'GB': 1e9, 'MB': 1e6, 'kB': 1e3, 'B': 1}  # 这只是一个例子，你可能需要根据实际情况修改

    # 转换所有的存储大小到统一的单位（这里我们选择的是字节）
    scan_costs['rows_size'] = scan_costs.apply(lambda row: row['rows_size'] * unit_conversion[row['rows_unit']], axis=1)

    # 计算所有存储大小的总和
    total_size = scan_costs['rows_size'].sum()

    # 打印结果
    print(round(total_size / 1024 / 1024 / 1024, 2))

In [49]:
with open(r"../workload/clickbench.txt", "r") as f:
    queries = [line.strip() for line in f]
for query in queries[24:]:
    calculate_cost(query, cursor)

0.53
0.44
0.53
70.26
1.13
1.13
2.51
1.21
2.54
2.51
1.21
1.21
1.21
5.47
0.95
0.95


In [35]:
# 提取所有Scan操作的存储大小，并分解为数量和单位
scan_costs = df[df.iloc[:, 0].str.contains('Scan')].iloc[:, 0].str.extractall(r'Estimates: \{rows: [0-9]* \((?P<rows_size>[0-9\.]*)(?P<rows_unit>[A-Za-z]*)\)')

# 转化数量到对应的数字
scan_costs['rows_size'] = scan_costs['rows_size'].astype(float)

# 创建一个字典用于存储每种单位的转换系数
unit_conversion = {'GB': 1e9, 'MB': 1e6, 'kB': 1e3, 'B': 1}  # 这只是一个例子，你可能需要根据实际情况修改

# 转换所有的存储大小到统一的单位（这里我们选择的是字节）
scan_costs['rows_size'] = scan_costs.apply(lambda row: row['rows_size'] * unit_conversion[row['rows_unit']], axis=1)

# 计算所有存储大小的总和
total_size = scan_costs['rows_size'].sum()

# 打印结果
print(total_size / 1024 / 1024 / 1024)

6.518282542936504
